# Higher Order Binary Optimization (HOBO) problems with qubovert

If `qubovert` is not `pip` installed and you are using these notebooks from within the `notebook_examples` directory, then run this cell. Otherwise it is not needed.

In [1]:
import sys
sys.path.append("..")

Import qubovert!

In [2]:
import qubovert

Create a Higher Order Binary Optimization (``HOBO``) object.

In [3]:
H = qubovert.HOBO()

Let's try to encode the optimization problem of finding the minimum of $C$, where

$$C = x_0 x_1 - 2x_1 + x_1 x_2 x_3,$$

In [4]:
H[('x_0', 'x_1')] += 1
H[('x_1',)] -= 2
H[('x_1', 'x_2', 'x_3')] += 1
# alternatively, can say H += {('x_0', 'x_1'): 1, ('x_1',): -2, ('x_1', 'x_2', 'x_3'): 1}
H

{('x_0', 'x_1'): 1, ('x_1',): -2, ('x_3', 'x_2', 'x_1'): 1}

An we subject $C$ to the constraints

$$x_0 \oplus x_1 = x_3, \quad 3x_0 + 2x_1 + 4x_3 < 7, \quad {\rm and} \quad x_0 x_1 = x_3 - x_2.$$

To enforce the constraints, we will pick a symbol $\lambda$ that we can tune as we like later.

In [5]:
#!pip install sympy
from sympy import Symbol

lam = Symbol("lam", real=True, positive=True)

H.XOR_eq(
    'x_0', 'x_1', 'x_3',
    lam=lam, constraint=True
    # see the docstring for more info on the `constraint` keyword argument, it is only 
    # necessary to add here if we plan on using the `H.solve_bruteforce` method.
).add_constraint_lt_zero(
    {('x_0',): 3, ('x_1',): 2, ('x_3',): 4, (): -7},
    lam=lam
).add_constraint_eq_zero(
    {('x_0', 'x_1'): 1, ('x_3',): -1, ('x_2',): 1}, 
    lam=lam
)
H

{('x_0', 'x_1'): 11*lam + 1,
 ('x_1',): -19*lam - 2,
 ('x_3', 'x_2', 'x_1'): 1,
 ('x_0',): -26*lam,
 ('x_3', 'x_0'): 22*lam,
 ('x_3', 'x_1'): 14*lam,
 ('x_3', 'x_0', 'x_1'): 2*lam,
 ('x_3',): -30*lam,
 ('x_0', '_a0'): 6*lam,
 ('x_0', '_a1'): 12*lam,
 ('x_0', '_a2'): 24*lam,
 ('x_1', '_a0'): 4*lam,
 ('x_1', '_a1'): 8*lam,
 ('_a2', 'x_1'): 16*lam,
 ('x_3', '_a0'): 8*lam,
 ('x_3', '_a1'): 16*lam,
 ('x_3', '_a2'): 32*lam,
 (): 36*lam,
 ('_a0',): -11*lam,
 ('_a1',): -20*lam,
 ('_a2',): -32*lam,
 ('_a0', '_a1'): 4*lam,
 ('_a2', '_a0'): 8*lam,
 ('_a2', '_a1'): 16*lam,
 ('x_2', 'x_0', 'x_1'): 2*lam,
 ('x_3', 'x_2'): -2*lam,
 ('x_2',): lam}

Notice that the constraints are automatically added to the objective function, and in particular the inequality constraint introduced some ancilla bits (labeled `'_a0'`, `'_a1'`, and `'_a2'`). Let's look at the constraints.

In [6]:
print(H.constraints)

{'eq': [{('x_0',): 1, ('x_1',): 1, ('x_0', 'x_1'): -2, ('x_3',): -1}, {('x_0', 'x_1'): 1, ('x_3',): -1, ('x_2',): 1}], 'lt': [{('x_0',): 3, ('x_1',): 2, ('x_3',): 4, (): -7}]}


Notice there are two equality constraints and one inequality constraint. The first equality one comes from enforcing the XOR constraint, and the second comes from enforcing the other equality constraint. The `'eq'` key of the constraints dictionary indicates that the quantity equals zero, and the `'lt'` key of the constraints dictionary indicates that the quantity is less than zero. Other possible keys are `'le'`, `'gt'`, and `'ge'`. See the docstrings for `HOBO.add_constraint_eq_zero`, `HOBO.add_constraint_lt_zero`, `HOBO.add_constraint_le_zero`, `HOBO.add_constraint_gt_zero`, and `HOBO.add_constraint_ge_zero` for info.

For testing purposes, let's solve this bruteforce to make sure everything is working.

In [7]:
solutions = H.solve_bruteforce(all_solutions=True)
print(solutions)

[{'x_0': 0, 'x_1': 1, 'x_2': 1, 'x_3': 1, '_a0': 0, '_a1': 0, '_a2': 0}]


Notice that there is one unique solution that minimizes the objective function and obeys all the constraints. We can get rid of the ancilla information with ``H.remove_ancilla_from_solution``.

In [8]:
solution = solutions[0]
H.remove_ancilla_from_solution(solution)

{'x_0': 0, 'x_1': 1, 'x_2': 1, 'x_3': 1}

Now let's solve this problem with a generic QUBO solver. Notice that the degree of problem is more than two, making `H` not a natural Quadratic Unconstrained Binary Optimization Problem (QUBO).

In [9]:
H.degree

3

We can convert it to a QUBO (note that there are some options for the reduction from PUBO to QUBO, see the `H.to_qubo` method for details). Ancilla bits will need to be added, and bit labels are mapped to integers.

In [10]:
Q = H.to_qubo()
print("num PUBO variables", H.num_binary_variables)
print("num QUBO variables", Q.num_binary_variables)
print()
print(Q)

num PUBO variables 7
num QUBO variables 9

{(0, 1): 15*lam + 3, (1,): -19*lam - 2, (7,): 6, (1, 2): 2, (1, 7): -4, (2, 7): -4, (3, 7): 1, (0,): -26*lam, (0, 3): 22*lam, (1, 3): 14*lam, (8,): 12*lam + 6, (0, 8): -8*lam - 4, (1, 8): -8*lam - 4, (3, 8): 2*lam, (3,): -30*lam, (0, 4): 6*lam, (0, 5): 12*lam, (0, 6): 24*lam, (1, 4): 4*lam, (1, 5): 8*lam, (1, 6): 16*lam, (3, 4): 8*lam, (3, 5): 16*lam, (3, 6): 32*lam, (4,): -11*lam, (5,): -20*lam, (6,): -32*lam, (4, 5): 4*lam, (4, 6): 8*lam, (5, 6): 16*lam, (2, 8): 2*lam, (2, 3): -2*lam, (2,): lam, (): 36*lam}


For testing purposes, let's solve this with bruteforce to see what the proper value of $\lambda$ should be to enforce the constraints. Notice how we remap the QUBO solution to the HOBO solution with `H.convert_solution(x)`.

In [11]:
for l in (1, 2, 3):
    Q_temp = Q.subs({lam: l})
    solutions = Q_temp.solve_bruteforce(all_solutions=True)
    solutions = [H.convert_solution(x) for x in solutions]
    print('lam', l)
    for s in solutions:
        print("\t", s, "is", "valid" if H.is_solution_valid(s) else "invalid")
    print()

lam 1
	 {'x_0': 0, 'x_1': 1, 'x_2': 0, 'x_3': 0, '_a0': 0, '_a1': 0, '_a2': 1} is invalid
	 {'x_0': 0, 'x_1': 1, 'x_2': 0, 'x_3': 1, '_a0': 0, '_a1': 0, '_a2': 0} is invalid
	 {'x_0': 0, 'x_1': 1, 'x_2': 1, 'x_3': 1, '_a0': 0, '_a1': 0, '_a2': 0} is valid

lam 2
	 {'x_0': 0, 'x_1': 1, 'x_2': 1, 'x_3': 1, '_a0': 0, '_a1': 0, '_a2': 0} is valid

lam 3
	 {'x_0': 0, 'x_1': 1, 'x_2': 1, 'x_3': 1, '_a0': 0, '_a1': 0, '_a2': 0} is valid



We see that $\lambda = 2$ is sufficient to enforce the constraints. So let's update our QUBO.

In [12]:
Q_good = Q.subs({lam: 2})

Now let's solve the QUBO with D'Wave's simulated annealer.

In [13]:
#!pip install dwave-neal
from neal import SimulatedAnnealingSampler

sampler = SimulatedAnnealingSampler()

Note that their software package takes in a specific form for QUBOs, namely, the keys of the dictionary must be two element tuples. This form can be accessed from `Q` and `Q_good` with `Q.Q` or `Q_good.Q`.

In [14]:
qubo_sample = sampler.sample_qubo(Q_good.Q, num_reads=100)
print("objective function:", qubo_sample.first.energy + Q_good.offset, "\n")

qubo_solution = qubo_sample.first.sample
print("qubo solution:", qubo_solution, "\n")

solution = H.convert_solution(qubo_solution)
print("hobo solution:", solution)
print("objective function:", H.value(solution), "\n")

print("The solution is", "valid" if H.is_solution_valid(solution) else "invalid")

objective function: -1.0 

qubo solution: {0: 0, 1: 1, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0} 

hobo solution: {'x_0': 0, 'x_1': 1, 'x_2': 1, 'x_3': 1, '_a0': 0, '_a1': 0, '_a2': 0}
objective function: -1 

The solution is valid


This matches the result of `H.solve_bruteforce()`. Now we'll solve an Ising formulation of our problem. Again we'll take $\lambda = 2$.

In [15]:
L = H.to_ising().subs({lam: 2})
# note that we cannot do H.subs({lam: 2}).to_ising()!! This is because H.subs({lam: 2})
# creates a new HOBO object, and it's mapping from variables labels to integers may be
# different than H's mapping. For example, try H.mapping == H.subs({lam: 2}).mapping a
# few times. They will often be different.
print("num PUBO variables", H.num_binary_variables)
print("num Ising variables", L.num_binary_variables)
print()
print(L)

num PUBO variables 7
num Ising variables 9

{(0, 1): 8.25, (0,): 9.25, (1,): 3.75, (): 45.0, (7,): 1.25, (1, 2): 0.5, (2,): 0.5, (1, 7): -1.0, (2, 7): -1.0, (3, 7): 0.25, (3,): 16.25, (0, 3): 11.0, (1, 3): 7.0, (8,): 7.0, (0, 8): -5.0, (1, 8): -5.0, (3, 8): 1.0, (0, 4): 3.0, (0, 5): 6.0, (0, 6): 12.0, (6,): 16.0, (1, 4): 2.0, (1, 5): 4.0, (1, 6): 8.0, (3, 4): 4.0, (3, 5): 8.0, (3, 6): 16.0, (4, 5): 2.0, (4, 6): 4.0, (4,): 4.0, (5, 6): 8.0, (5,): 8.0, (2, 8): 1.0, (2, 3): -1.0}


Similar to their QUBO solver, D'Wave's Ising solver accepts a specific form for Ising models, namely a linear term dictionary and a quadratic term dictionary. These can be accessed with `L.h` and `L.J`.

In [16]:
ising_sample = sampler.sample_ising(L.h, L.J, num_reads=100)
print("objective function:", ising_sample.first.energy + L.offset, "\n")

ising_solution = ising_sample.first.sample
print("ising solution:", ising_solution, "\n")

solution = H.convert_solution(ising_solution)
print("hobo solution:", solution)
print("objective function:", H.value(solution), "\n")

print("The solution is", "valid" if H.is_solution_valid(solution) else "invalid")

objective function: -1.0 

ising solution: {0: -1, 1: 1, 2: 1, 3: 1, 4: -1, 5: -1, 6: -1, 7: 1, 8: -1} 

hobo solution: {'x_0': 0, 'x_1': 1, 'x_2': 1, 'x_3': 1, '_a0': 0, '_a1': 0, '_a2': 0}
objective function: -1 

The solution is valid


Again this matches the result of `H.solve_bruteforce()`.